# Assignment 2

This is the first of two mandatory assignments which must be completed during the course. First some practical information:

* When is the assignment due?: **23:59, Friday, August 24, 2018**
* How do you grade the assignment?: You will **peergrade** each other as primary grading. 
* Must I hand-in as a group?: **yes**

The assigment consist of one to three problems from each of the exercise sets you have solved so far (excluding Exercise Set 1). We've tried to select problems which are self contained, but it might be nessecary to solve some of the previous exercises in each set to fully answer the problems in this assignment.

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

## Problems from exercise set 11

> **_Hint:_** you can get data by running 

```python
iris = sns.load_dataset('iris')
iris = iris.query("species == 'virginica' | species == 'versicolor'").sample(frac = 1, random_state = 3)
X = np.array(iris[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']])
y = np.array(iris['species'].map({'virginica': 1, 'versicolor': -1}))

# A very simple deterministic test-train split 
Xtrain = X[:70]
ytrain = y[:70]

Xtest = X[70:]
ytest = y[70:]
```

> The following code gives you the required functions to work with:

```python
def random_weights(location = 0.0, scale = 0.01, seed = 1):
    # Init random number generator
    rgen = np.random.RandomState(seed)
    w = rgen.normal(loc=location, scale=scale, size= 1 + X.shape[1])    
    return w

def net_input(X, W): 
    return np.dot(X, W[1:]) + W[0]   # Linear product X'W + bias


def predict(X, W):
    linProd = net_input(X, W)
    return np.where(linProd >= 0.0, 1, -1)    # 1(linProd > 0)
```


> **Ex. 11.1.5:** Write a function whichs loops over the training data (both X and y) using `zip`. For each row in the data, update the weights according to the perceptron rule (remember to update the bias in `w[0]`!). Set $\eta = 0.1$.
>
> Make sure the loop stores the total number of prediction errors encountered underways in the loop by creating an int which is incremented whenever you update the weights. 
>
>> _Hint:_ your function should return the updated weights, as well as the number of errors made by the perceptron.
>
>> _Hint:_ The following code block implements the function in _pseudo_code (it wont run, but serves to communicate the functionality).
>> ```
>> function f(X, y, W, eta):
>>    set errors = 0
>>
>>    for each pair xi, yi in zip(X,y) do:
>>        set update = eta * (yi - predict(xi, W))
>>        set W[1:] = W[1:] + update * xi
>>        set W[0] = W[0] + update
>>        set errors = errors + int(update != 0) 
>>
>>    return W, errors
>> ```
>
> *Bonus:* If you completed the previous bonus assignment, calculate the accuracy on training data using the updated weights. Any progress yet?

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

In [3]:
iris = sns.load_dataset('iris')
iris = iris.query("species == 'virginica' | species == 'versicolor'").sample(frac = 1, random_state = 3)
X = np.array(iris[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']])
y = np.array(iris['species'].map({'virginica': 1, 'versicolor': -1}))

# A very simple deterministic test-train split 
Xtrain = X[:70]
ytrain = y[:70]

Xtest = X[70:]
ytest = y[70:]

#The following code gives you the required functions to work with:

def random_weights(location = 0.0, scale = 0.01, seed = 1):
    # Init random number generator
    rgen = np.random.RandomState(seed)
    w = rgen.normal(loc=location, scale=scale, size= 1 + X.shape[1])    
    return w

def net_input(X, W): 
    return np.dot(X, W[1:]) + W[0]   # Linear product X'W + bias



def predict(X, W):
    linProd = net_input(X, W)
    return np.where(linProd >= 0.0, 1, -1)    # 1(linProd > 0)

In [4]:
W = random_weights()
z = net_input(Xtrain, W)
y_hat = predict(Xtrain,W)

In [5]:
# [Answer to Ex. 11.1.5]
def f(X, y, W, eta):
    errors = 0
    
    for xi, yi in zip(Xtrain, y):
        bias = eta * (yi - predict(xi, W))
        W[1:] = W[1:] + bias * xi
        W[0] = W[0] + bias
        errors = errors + int(bias != 0)
    
    return W, errors

eta = 0.1
f(Xtrain, y, W, eta)

(array([-0.58375655, -1.56611756, -0.76528172,  1.00927031,  1.36865408]), 33)

> **Ex. 11.1.6:** Write a function which repeat the updating procedure you constructed in 11.1.5 for `n_iter` times by packing the whole thing in an outer loop. Make sure you store the number of errors in each iteration in a list. 
>
> Plot the total errors after each iteration in a graph.
>
>> _Hint:_ Make sure you dont reset the weights after each iteration.
>
>> _Hint:_ Once again some pseudocode:
>> ```
>> function g(X, y, n_iter):
>>     set eta = 0.1
>>     set weights = random_weights()
>>     set errorseq = list()
>>
>>     for each _ in range(n_iter):
>>         weights, e = f(X, y, W, eta) 
>>         errorseq.append(e)
>>
>>     return weights, errorseq
>> ```
>
>> _Bonus:_ Wrap the code in a function called Perceptron.

In [ ]:
# [Answer to Ex. 11.1.6]
def Perceptron(X, y, n_iter):
    eta = 0.1
    W = random_weights()
    errorseq = list()
    
    for i in range(n_iter):
        w, e = f(X,y,W,eta)
        errorseq.append(e)
    plt.xlabel("Number of iterations")
    plt.ylabel("Number of errors")
    return print("Weight vector: " + str(W)), \
            plt.plot(errorseq), \
            print("First 10 errors: " + str(errorseq[:10])), \
            print("Last 10 errors: " + str(errorseq[-10:]))

## Problems from exercise set 12
>Get the required data by running 

```python
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

cal_house = fetch_california_housing()    
X = pd.DataFrame(data=cal_house['data'], 
                 columns=cal_house['feature_names'])\
             .iloc[:,:-2]
y = cal_house['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.5, random_state=1)
```

> **Ex. 12.2.0:** Load the california housing data with scikit-learn using the code above. Inspect the data set. 

In [3]:
# [Answer to Ex. 12.2.0]
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

cal_house = fetch_california_housing()    
X = pd.DataFrame(data=cal_house['data'], 
                 columns=cal_house['feature_names'])\
             .iloc[:,:-2]
y = cal_house['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.5, random_state=1)

> **Ex.12.2.1**: Generate interactions between all features to third degree, make sure you **exclude** the bias/intercept term. How many variables are there? Will OLS fail? 
>
> After making interactions rescale the features to have zero mean, unit std. deviation. Should you use the distribution of the training data to rescale the test data?  
>
>> *Hint 1*: Try importing `PolynomialFeatures` from `sklearn.preprocessing`
>
>> *Hint 2*: If in doubt about which distribution to scale, you may read [this post](https://stats.stackexchange.com/questions/174823/how-to-apply-standardization-normalization-to-train-and-testset-if-prediction-i).

In [42]:
# [Answer to Ex. 12.2.1]
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

#The third degree interactions are defined in X_pol based on the TRAINING dataset
X_pol = PolynomialFeatures(degree = 3, interaction_only=True, include_bias=False).fit_transform(X_train)
# and contain 10320 variables (holy moly..)
print(len(X_pol))

#The interaction features are rescaled - we only rescale the TRAIN dataset.
X_train_rescale = StandardScaler(X_train)
print(X_train_rescale)

10320
StandardScaler(copy=       MedInc  HouseAge   AveRooms  AveBedrms  Population  AveOccup
10089  4.0893      35.0   5.267760   0.983607      1056.0  2.885246
2136   3.7578      24.0   5.061538   0.957692       781.0  3.003846
17546  2.4306      39.0   4.899209   1.069170      1990.0  3.932806
10051  3.2813      10.0...773585
235    2.3036      35.0   4.620513   1.176923      1009.0  2.587179

[10320 rows x 6 columns],
        with_mean=True, with_std=True)


> **Ex.12.2.2**: Estimate the Lasso model on the train data set, using values of $\lambda$ in the range from $10^{-4}$ to $10^4$. For each $\lambda$  calculate and save the Root Mean Squared Error (RMSE) for the test and train data. 
>
>> *Hint*: use `logspace` in numpy to create the range.


In [41]:
# [Answer to Ex. 12.2.2]

lambdas = np.logspace(-4, 4, 12)


> **Ex.12.2.3**: Make a plot with on the x-axis and the RMSE measures on the y-axis. What happens to RMSE for train and test data as $\lambda$ increases? The x-axis should be log scaled. Which one are we interested in minimizing? 

> Bonus: Can you find the lambda that gives the lowest MSE-test score?

In [ ]:
# [Answer to Ex. 12.2.3]

## Problems from exercise set 13

> **Ex. 13.1.3:**
Run a Lasso regression using the Pipeline from `Ex 13.1.2`. In the outer loop searching through the lambdas specified below. 
In the inner loop make 5 fold cross validation on the selected model and store the average MSE for each fold. Which lambda gives the lowest test MSE?
>
> ```python 
lambdas =  np.logspace(-4, 4, 12)
```
>
>> *Hint:* `KFold` in `sklearn.model_selection` may be useful.
>
> This code will give you the required data: 

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold


import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)


import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import seaborn as sns

cal_house = fetch_california_housing()    
X = pd.DataFrame(data=cal_house['data'], 
                 columns=cal_house['feature_names'])\
             .iloc[:,:-2]
y = cal_house['target']

In [10]:
# [Answer to Ex. 13.1.3]
#split data into development and test 
X_dev, X_test, y_dev, y_test = train_test_split(X, y, test_size=1/3, random_state=1)

#split development data into train & validation 
X_train, X_val, y_train, y_val = train_test_split(X_dev, y_dev, test_size=1/2, random_state=1)

X_train_index = X_train.index
X_val_index = X_val.index
#y_train_index = y_train.index
#y_val_index = y_val.index


In [13]:
kfolds = KFold(n_splits=5) #splits the data into 5 equally large splits 
lambdas = np.logspace(-4, 4, 12)

for lambda_ in lambdas:
    
    pipe_lassoCV = make_pipeline(PolynomialFeatures(degree=3, include_bias=False),
                                 StandardScaler(),
                                 Lasso(alpha=lambda_, random_state=1))    
    mseCV_ = []
    
    for train_index, val_index in kfolds.split(X_dev, y_dev):
        
        X_train,  X_val = X_dev.loc[X_train_index], X_dev.loc[X_val_index]
        y_train, y_val = y_dev.loc[X_train_index], y_dev.loc[y_val_index]
        
        #X_train, y_train, = X_dev[train_idx], y_dev[train_idx]
        #X_val, y_val = X_dev[val_idx], y_dev[val_idx] 

        pipe_lassoCV.fit(X_train, y_train)
        
        mseCV_.append(mse(pipe_lassoCV.predict(X_val), y_val)) 
        hyperparam_perform = pd.Series (mseCV_, index=lamdas)
        #optimal = pd.Series(mseCV_, index=lambdas).nsmallest(1)
    
    mseCV.append(mseCV_)
    
#something's wrong with the indices 

NameError: name 'lamdas' is not defined

## Problems from exercise set 14

> **Ex. 14.1.3**: Train a decision tree classifier on **all** of the titanic data.
>
>* If your feature matrix is `X` and your target array is `y` you can do this by instantiating a model like:
>
        from sklearn.tree import DecisionTreeClassifier
        model = DecisionTreeClassifier()
        model.fit(X, y)  # <--- This is the training/fitting/learning step
>       
>Write four functions that counts the number of ..
 - true positives where we call the function `TP`;
 - true negatives where we call the function `TN`;
 - false positives where we call the function `FP`;
 - false negatives where we call the function `FN`.

> All of these functions should take three arguments, the actual y column, the actual X column and a fittedModel object (e.g. `fittedModel = DecisionTreeClassifier.fit(X,y)`)
>
>> _Hint 1:_ use the function `np.where` to compare y and the predicted values. For example `y + prediction == 2` is true only for the True Positives.
>
>> _Hint 2:_ You can check if your result is correct by summing all four functions and checking that the result equals to the number of observations. 
>
> Get the data by running:

In [11]:
rawdata = sns.load_dataset('titanic').sample(frac=1, random_state = 3)

# Feature creation
rawdata['male'] = (rawdata['sex'] == 'male').astype(int)
rawdata['alone'] = rawdata['alone'].astype(int)
rawdata['adult_male'] = rawdata['adult_male'].astype(int)


# The following piece of code (in multiple lines) generates 
# dummies for all of the categorical variables. 
data = pd.get_dummies(rawdata, 
                      columns = ['class', 'sibsp', 'parch', 'deck'], 
                      drop_first=True)\
         .drop(['pclass', 'sex', 'embarked', 'who', 'embark_town', 'alive'], axis = 1)


data = data.dropna().reset_index()

X = np.array(data.drop('survived', axis = 1))
y = np.array(data['survived'])

In [ ]:
# [Answer to Ex 14.1.3]

> **Ex. 14.1.4:** Combine the four functions you defined above to write your own accuracy function, which calculates
$$
ACC = \frac{TP + TN}{TP+ TN + FP + FN}
$$
>
> Test the accuracy of your model using the `A` function. Report the accuracy of your model on the same data that you trained the model on.
>
>> _Note:_ The reason we want to split the calculation of accuracy into these four components, is that we can then easily calculate other scores, such as the _precision, recall and f1_.

In [ ]:
# [Answer to Ex 14.1.4]

## Problems from exercise set 15 

> **Ex. 15.1.1:** 
Import the Counter object from the builtin package collections (Hint1). This is essentially a dictionary designed for keeping counts, same syntax, but extra functionality. We don't have to initialize each key. We can write: 

```python
c = Counter()
# then we can do this
c['hej']+=1
# without first defining c['hej'] = 0
```


>* Initialize a Counter object and assign it to the variable `dc` (document count).
>* Define a list named `text_counts`. In this container we will store each document after we have converted it to counts of tokens.
>* Run through all tokenized texts and
    * initialize a Counter object with the tokenized text as input, assign this object to a variable `c_t`. >This will now contain a count of each token in the document. Append `c_t` to our list `text_counts`.
    * run though each key in the `c_t` and increment the document count variable `dc` by one. (Hint2)

(hint1: from ... import ...)

(hint2: dc[token]+=1)

> Run the following code to get the data:

In [3]:
import nltk
nltk.download('punkt')
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/snorreralund/scraping_seminar/master/english_review_sample.csv')
import re
token_re = re.compile('\w+')

tokenized = df.reviewBody.apply(nltk.word_tokenize)

tokenized

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Naeim.S\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


0       [Lots, of, inventory, ,, very, fast, and, effi...
1       [I, did, not, received, the, map, I, had, orde...
2       [After, searching, a, number, of, stores, here...
3       [Website, is, not, intuitive, ., I, do, n't, l...
4       [Outstanding, customer, service, ,, appreciate...
5       [Waiting, in, line, to, get, on, the, Big, Bus...
6       [I, 've, been, to, the, 'other, ', sac, store,...
7       [I, do, n't, even, bother, making, hotel, rese...
8                                      [Great, packaging]
9       [They, hurt, like, crazy, .., not, worth, the,...
10      [We, were, told, the, dentist, could, Do, seda...
11                         [Very, Good, Company, :, -, )]
12      [Helpful, ,, friendly, and, did, n't, give, me...
13      [Delivered, very, quickly, and, exactly, what,...
14      [Very, nice, calendars, ,, cards, ,, and, stat...
15      [I, had, to, move, my, car, from, Dublin, ,, O...
16      [the, signs, were, made, very, well, ,, very, ...
17      [One, 

In [25]:
# [Answer to Ex. 15.1.1]
from collections import Counter
dc = Counter()
text_counts = []

for text in tokenized:
    c_t = Counter(text)
    text_counts.append(c_t)
    
    for token in c_t:
        dc[token] += 1

In [26]:
#Let's inspect it:
print(text_counts[0:2])
dc
#Looks' good.

[Counter({'.': 2, 'Lots': 1, 'of': 1, 'inventory': 1, ',': 1, 'very': 1, 'fast': 1, 'and': 1, 'efficient': 1, 'I': 1, 'would': 1, 'recommend': 1, 'this': 1, 'company': 1}), Counter({'and': 4, '.': 4, 'was': 4, 'to': 4, 'I': 3, 'did': 3, 'map': 3, 'me': 3, 'the': 2, 'emailed': 2, 'an': 2, 'review': 2, 'my': 2, 'a': 2, 'not': 1, 'received': 1, 'had': 1, 'ordered': 1, 'paid': 1, 'for': 1, 'within': 1, 'stated': 1, 'delivery': 1, 'time': 1, 'Mapscompany': 1, 'their': 1, 'only': 1, 'first': 1, 'response': 1, 'send': 1, 'invitation': 1, 'experience': 1, '!': 1, 'My': 1, 'initial': 1, 'therefore': 1, 'extremely': 1, 'negative': 1, 'Since': 1, 'then': 1, 'support': 1, 'team': 1, 'member': 1, 'reassuring': 1, 'but': 1, 'ultimately': 1, "n't": 1, 'provide': 1, 'with': 1, 'any': 1, 'concrete': 1, 'information': 1, 'as': 1, 'where': 1, 'The': 1, 'eventually': 1, 'arrive': 1, 'is': 1, 'great': 1, ',': 1, 'so': 1, 'good': 1, 'end': 1, 'imperfect': 1, 'start': 1})]


Counter({'Lots': 20,
         'of': 2604,
         'inventory': 20,
         ',': 4012,
         'very': 1820,
         'fast': 685,
         'and': 6214,
         'efficient': 139,
         '.': 7546,
         'I': 5112,
         'would': 1163,
         'recommend': 773,
         'this': 1350,
         'company': 721,
         'did': 752,
         'not': 1360,
         'received': 562,
         'the': 5076,
         'map': 3,
         'had': 1368,
         'ordered': 684,
         'paid': 177,
         'for': 2887,
         'within': 212,
         'stated': 48,
         'delivery': 475,
         'time': 1242,
         'emailed': 51,
         'Mapscompany': 1,
         'their': 581,
         'only': 533,
         'first': 428,
         'response': 144,
         'was': 3409,
         'to': 4822,
         'send': 142,
         'me': 1384,
         'an': 860,
         'invitation': 7,
         'review': 101,
         'my': 2774,
         'experience': 678,
         '!': 2657,
         'My

> **Ex. 15.1.2:** 
Now we define the the inverse document frequency variable `idf` as a dictionary with the tokens as keys and idf weights as values. We do this by running through both the token and the value (document count) in the `dc` variable and calculate the ratio between number documents and the token document counts. 

>Use the `np.log` function for the log transform.

>We can iterate through this using the `.items()` syntax we know from the dictionary. 


In [27]:
# [Answer to Ex. 15.1.2]
import numpy as np
n_docs = len(tokenized)

#Using a loop:
# idf = {}
# for k, v in dc.items():
#     idf[k] = np.log(n_docs / value)

#Using an smooth oneliner:
idf = {key:np.log(n_docs / val) for key,val in dc.items()}

In [28]:
#Let's inspect the Inverse Document Frequency:
idf
#Look's good!

{'Lots': 6.214608098422191,
 'of': 1.3455363686477229,
 'inventory': 6.214608098422191,
 ',': 0.9132952228943566,
 'very': 1.7037485919053417,
 'fast': 2.6809215337139576,
 'and': 0.47578028202319006,
 'efficient': 4.275866438845491,
 '.': 0.28156747145192695,
 'I': 0.6709943759188121,
 'would': 2.1515822194575183,
 'recommend': 2.560061323388761,
 'this': 2.0024805005437076,
 'company': 2.6297012346912334,
 'did': 2.5876040480263427,
 'not': 1.995100393246085,
 'received': 2.8788385220824915,
 'the': 0.6780615431419045,
 'map': 8.111728083308073,
 'had': 1.989235273793687,
 'ordered': 2.682382454353632,
 'paid': 4.034190639402354,
 'for': 1.2423671923132475,
 'within': 3.8537540973041704,
 'stated': 5.339139361068292,
 'delivery': 3.0470255679415414,
 'time': 2.0858621094827585,
 'emailed': 5.278514739251857,
 'Mapscompany': 9.210340371976184,
 'their': 2.8455896151242714,
 'only': 2.931818947810338,
 'first': 3.151217176394386,
 'response': 4.240527072400182,
 'was': 1.07616609983827

> **Ex. 15.1.3:** 
Now we weight the term frequency in each document with the idf value of each token. Here we used our `text_counts` variable that almost holds the frequency, we just need to divide by the number of tokens in the document. 
Define a list container: `tfidf_docs`. 

FIRST LOOP: For each counter in the text_count container:
    * define the variable `doc_n` as sum of all values in the counter - `.values()` .
    * define a dictionary named `tfidf`.
    * SECOND LOOP: run through all tokens, and their counts by using the `.items()` method of the counter.
        * define a value tf as the ratio between the count and the sum.
        * now weight this value with the idf weight found by calling the idf variable with the token as key.
        * assign this weighed term frequency to the tfidf[token].
    * Once outside the second loop. Append the tfidf dictionary to the tfidf_docs list container.

In [ ]:
# [Answer to Ex. 15.1.3]
tfidf_docs = list()
for object in text_counts:
    doc_n = sum(object.values())
    tfidf = {}
    
    for token, counter in dc.items():
        tf = counter / doc_n
        tfidf[token] = tf * idf[token] 
    
    tfidf_docs.append(tfidf)

In [ ]:
#Let's inspect the Term Frequency - Inverse Document Frequency:
tfidf_docs[0:2]


## Problems from exercise set 16


> **Ex. 16.2.5:** You should now implement it by doing the following:
* Define a maximum number of iterations`max_iter` to 15.
* Use the `initialize_clusters` function to define a variable `centroids`.
* make a `for` loop from 0 to max_iter where you: 
    * copy the current cluster centroids to a new variable: old_centroids. This will be used for checking convergence after the maximization step.
    * define the `cluster_assignment`  by running the `maximize` function
    * define a new (i.e. overwrite) `centroids` variable by running the `update_expectation` function.
    * finally check if old_centroids is equal to new_centroids, using the np.array_equal() function. If they are: break.

Make sure that it works and wrap it around a function `fit_transform()` that takes the data `X` as input, and the number of clusters `k` plus the maximum number of iterations `max_iter`. It should return the cluster assignments and the cluster centroids. 


> The code below will give you the dataset, as well as three functions you need to solve the exercise.

In [27]:
import seaborn as sns 
import pandas as pd 
import random
from sklearn.metrics import pairwise_distances

df = sns.load_dataset('iris')
X = df[['sepal_length','sepal_width','petal_length','petal_width']].values


def initialize_clusters(k,X):
    idx = random.sample(range(len(X)),k)
    centroids = X[idx]
    return centroids


def maximize(centroids,X):
    dist_matrix = pairwise_distances(centroids,X)
    cluster_assignment = dist_matrix.T.argsort(axis=1)[:,0]
    return cluster_assignment


def update_expectation(k,X,cluster_assignment):
    new_centroids = np.zeros((k,len(X[0])))
    for i in range(k):
        subset = X[cluster_assignment==i]
        new_centroids[i] = subset.mean(axis=0)
    return new_centroids


In [ ]:
# [Answer to Ex. 16.2.5]